# Unsupervised Learning: Association Rule Mining


by [__Michael Granitzer__ (michael.granitzer@uni-passau.de)]( http://www.mendeley.com/profiles/michael-granitzer/) and [Konstantin Ziegler (konstantin.ziegler@uni-passau.de)](http://zieglerk.net) based on examples from the [scikit-learn documentation](http://scikit-learn.org/stable/)

__License__

This work is licensed under a [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

1. Turn the pseudo-code for the A-priori algorithm from the lecture into a quick-and-dirty implementation. (Efficiency is not crucial, because the dataset is not that large. Correctness is. An implementation with python lists should suffice, see the end of the notebook.)
2. Use your algorithm from 1. on the dataset to find frequent itemsets. 
Note: The algorithm from the lecture runs only on binary features; So, you have to turn the numerical features into binaries, e.g. by comparison with the average.
Note: This requires the choice of a suitable minimum support; in practice you don't know "suitable" in advance, so binary search the possible values for sigma until the number of frequent itemsets is easy for you to examine by hand.
3. Bonus: Use the frequent itemsets of 2. to find strong association rules. Note: Analogously to 2., this requires the choice of a proper confidence. What is the best possible confidence that you can achieve?
4. Bonus: Filter your association rules for correlation rules, e.g. using lift as correlation measure.
5. Interpretation: Could the observed rule be an effect of a larger cause?

# Implementation of the Apriori-Algorithm

In [31]:
# Example from the Lecture, slide 8
names = ['milk', 'butter', 'coffee', 'cacao', 'cake', 'sugar', 'tea']
X = [[1, 1, 0, 0, 0, 0, 0],
     [1, 0, 1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1, 0, 0],
     [0, 0, 1, 0, 0, 1, 1],
     [1, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1, 1]]
# Some 2-itemsets
I1 = [0, 1]
I2 = [1, 5]
I3 = [0, 2]

In [32]:
x = X[1]
all([x[j] == 1 for j in I3])

True

In [33]:
# Return corresponding instance sets
def is_instance(x, I):
    return all([x[j] == 1 for j in I])

def all_instances(X, I):
    return [x for x in X if is_instance(x, I)]

print(all_instances(X, I1))
print(all_instances(X, I2))
print(all_instances(X, I3))

[[1, 1, 0, 0, 0, 0, 0]]
[]
[[1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0]]


In [34]:
def support(X, I):
    return len(all_instances(X, I))/len(X)
print(support(X, I1))
print(support(X, I2))
print(support(X, I3))


0.16666666666666666
0.0
0.3333333333333333


In [35]:
def is_frequent_itemset(X, L, sigma):
    return all([support(X, I) >= sigma for I in L])

print(all_instances(X, I1))
print(all_instances(X, I2))
print(all_instances(X, I3))

frequent_2_item_set_L2 = [I1, I2, I3]

print(is_frequent_itemset(X, frequent_2_item_set_L2, 1/6))

[[1, 1, 0, 0, 0, 0, 0]]
[]
[[1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0]]
False


In [36]:
def getLabels(itemset, labels):
    return [labels[i] for i in itemset]

def hasAprioriProperty(c, Llast):
    for item in c:
        d = c[:]
        d.remove(item)
        if d not in Llast:
            return False
    return True

def aPriori(X, sigma, verbose=True):
    L1 = [[i] for i in range(len(X[0])) if support(X, [i]) >= sigma]
    print(L1)
    L = [L1]
    for k in range(2, len(X[0]) + 1):
        if verbose:
            print('start loop with k=%d and L=%s' %(k, L))
        C = [sorted(a + b) for a in L[-1] for b in L1 if b[0] not in a]
        C = [c for i, c in enumerate(C) if c not in C[:i]]    # remove duplicates, because we use lists instead of sets
        if verbose:
            print('before Apriori-Check C=%s' %C)
        C = [c for c in C if hasAprioriProperty(c, L[-1])]
        if verbose:
            print('after Apriori-Check C=%s' %C)
        Csigma = dict([(tuple(c), 0.0) for c in C])
        for x in X:
            Ct = [c for c in C if is_instance(x, c)]
            for c in Ct:
                Csigma[tuple(c)] += 1/len(X)
        C = [c for c in C if Csigma[tuple(c)] >= sigma]
        if len(C) == 0:
            break
        L += [C]       
    return [c for Lk in L for c in Lk ]

print(aPriori(X, 0.3))
print(aPriori(X, 0.5))
print(aPriori(X, 0.7))    

[[0], [2], [4], [5], [6]]
start loop with k=2 and L=[[[0], [2], [4], [5], [6]]]
before Apriori-Check C=[[0, 2], [0, 4], [0, 5], [0, 6], [2, 4], [2, 5], [2, 6], [4, 5], [4, 6], [5, 6]]
after Apriori-Check C=[[0, 2], [0, 4], [0, 5], [0, 6], [2, 4], [2, 5], [2, 6], [4, 5], [4, 6], [5, 6]]
start loop with k=3 and L=[[[0], [2], [4], [5], [6]], [[0, 2], [0, 4], [2, 5], [5, 6]]]
before Apriori-Check C=[[0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 4, 5], [0, 4, 6], [2, 4, 5], [2, 5, 6], [0, 5, 6], [4, 5, 6]]
after Apriori-Check C=[]
[[0], [2], [4], [5], [6], [0, 2], [0, 4], [2, 5], [5, 6]]
[[0], [2], [5]]
start loop with k=2 and L=[[[0], [2], [5]]]
before Apriori-Check C=[[0, 2], [0, 5], [2, 5]]
after Apriori-Check C=[[0, 2], [0, 5], [2, 5]]
[[0], [2], [5]]
[]
start loop with k=2 and L=[[]]
before Apriori-Check C=[]
after Apriori-Check C=[]
[]


In [37]:
# Example from the board
names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
X     = [[1,   0,   0,   1,   1,   0,   0,   1,   0,   0,   0],
         [1,   0,   0,   1,   0,   1,   0,   0,   1,   1,   0],
         [0,   1,   1,   1,   1,   1,   1,   0,   0,   0,   0],
         [1,   1,   1,   1,   1,   1,   0,   0,   0,   0,   0],
         [0,   1,   0,   1,   1,   0,   0,   0,   0,   1,   1]]
L = aPriori(X, 3/5)
L_names = [getLabels(itemset, names) for itemset in L]
print(L)
print(L_names)

[[0], [1], [3], [4], [5]]
start loop with k=2 and L=[[[0], [1], [3], [4], [5]]]
before Apriori-Check C=[[0, 1], [0, 3], [0, 4], [0, 5], [1, 3], [1, 4], [1, 5], [3, 4], [3, 5], [4, 5]]
after Apriori-Check C=[[0, 1], [0, 3], [0, 4], [0, 5], [1, 3], [1, 4], [1, 5], [3, 4], [3, 5], [4, 5]]
start loop with k=3 and L=[[[0], [1], [3], [4], [5]], [[0, 3], [1, 3], [1, 4], [3, 4], [3, 5]]]
before Apriori-Check C=[[0, 1, 3], [0, 3, 4], [0, 3, 5], [1, 3, 4], [1, 3, 5], [0, 1, 4], [1, 4, 5], [3, 4, 5]]
after Apriori-Check C=[[1, 3, 4]]
start loop with k=4 and L=[[[0], [1], [3], [4], [5]], [[0, 3], [1, 3], [1, 4], [3, 4], [3, 5]], [[1, 3, 4]]]
before Apriori-Check C=[[0, 1, 3, 4], [1, 3, 4, 5]]
after Apriori-Check C=[]
[[0], [1], [3], [4], [5], [0, 3], [1, 3], [1, 4], [3, 4], [3, 5], [1, 3, 4]]
[['A'], ['B'], ['D'], ['E'], ['F'], ['A', 'D'], ['B', 'D'], ['B', 'E'], ['D', 'E'], ['D', 'F'], ['B', 'D', 'E']]


In [7]:
# Example from Lecture p. 21
names = ['I1', 'I2', 'I3', 'I4', 'I5']
X     = [[1,    1,    0,    0,    1],
         [0,    1,    0,    1,    0],
         [0,    1,    1,    0,    0],
         [1,    1,    0,    1,    0],
         [1,    0,    1,    0,    0],
         [0,    1,    1,    0,    0],
         [1,    0,    1,    0,    0],
         [1,    1,    1,    0,    1],
         [1,    1,    1,    0,    0]]

print(aPriori(X, 2/9))

[[0], [1], [2], [3], [4], [0, 1], [0, 2], [0, 4], [1, 2], [1, 3], [1, 4], [0, 1, 2], [0, 1, 4]]
